In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import neighbors
%matplotlib inline

import seaborn as sns; sns.set()

#ignore 'Future warnings'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [17]:
# Reading and viewing data
winedf =  pd.read_csv("./data/wine-reviews/winemag-data-130k-v2.csv")

#how many nulls
print(winedf.isnull().sum())



Unnamed: 0                   0
country                     63
description                  0
designation              37465
points                       0
price                     8996
province                    63
region_1                 21247
region_2                 79460
taster_name              26244
taster_twitter_handle    31213
title                        0
variety                      1
winery                       0
dtype: int64


In [42]:
#how does the data looks like
print(winedf.shape)
print("\n")
print(winedf.dtypes)


(129971, 14)


Unnamed: 0                 int64
country                   object
description               object
designation               object
points                     int64
price                    float64
province                  object
region_1                  object
region_2                  object
taster_name               object
taster_twitter_handle     object
title                     object
variety                   object
winery                    object
dtype: object


In [18]:
# imputing missing values

#imputing a constant 'unknown' for country and province
winedf.country = winedf.country.fillna('unknown')
winedf.province = winedf.province.fillna('unknown')

#imputing country-province combo for region_1 and region_2
winedf.region_1 = winedf.region_1.fillna((winedf.country + '-' + winedf.province))
winedf.region_2 = winedf.region_2.fillna((winedf.region_1))

#imputing mean value for price

winedf.price = winedf.groupby(['country','variety','region_1'])['price'].apply(lambda x: x.fillna(x.mean()))
winedf.price = winedf.groupby(['country'])['price'].apply(lambda x: x.fillna(x.mean()))
winedf.price.fillna(winedf.price.mean(), inplace=True)

winedf.variety =  winedf.variety.fillna(method='ffill')

winedf.designation = winedf.designation.fillna(winedf.title.str.extract('([^\d|(|)]+(?=[(]))'))
winedf.designation = winedf.designation.fillna(winedf.title.str.extract('([^\d|(|)]+(?=$))'))

winedf['vintage'] = winedf.title.str.extract('([\d]{4})')
winedf['vintage'] = winedf.vintage.fillna('2013')
winedf.vintage =  winedf.vintage.astype(np.int)

#no more na's
print(winedf.isnull().sum())


Unnamed: 0                   0
country                      0
description                  0
designation                  0
points                       0
price                        0
province                     0
region_1                     0
region_2                     0
taster_name              26244
taster_twitter_handle    31213
title                        0
variety                      0
winery                       0
vintage                      0
dtype: int64


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score



In [37]:
# Use TFIDF Tokenizer to create categorical columns of words from 
# the 'description' column and then apply Multinomical Naive Bayes classifier 
#to predict (impute) 'taster_name'
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Training data will the rows that have the 'taster_name'
Train = winedf.loc[winedf.taster_name.isna() == False, ['description', 'taster_name']]

# Test (prediction) dataset will be the rows with nas for 'taster_name'
Test = winedf.loc[winedf.taster_name.isna(), ['description', 'taster_name']]

# use cross validation to verify the prediction accuracy of the model
#cross_val_score(model,Train['description'],Train['taster_name'],cv=5 )

# Train the model and use it to predict the 'taster_name'
model.fit(Train['description'],Train['taster_name'])
taster_name_pred =  model.predict(Test['description'])

#from sklearn.metrics import accuracy_score
#accuracy_score(X['taster_name'], taster_name_pred)

# impute the 'taster_name' from the predicted labels from the model
winedf.loc[winedf.taster_name.isna(), ['taster_name']] = taster_name_pred

In [50]:
#Some interesting use of grouping function

print('give me list of countries with atleast 100 wine reviews')
print(winedf.groupby('country').filter(lambda x: x['price'].count() > 100)['country'].unique())

print('\n')
print('give me list of countries with above average wine price')
print(winedf.groupby('country').filter(lambda x: (x['price'].count() > 100) & (x['price'].mean() > winedf['price'].mean()))['country'].unique())


print('\n')
print('give me list of countries with above average wine rating')
print(winedf.groupby('country').filter(lambda x: (x['points'].count() > 100) & (x['points'].mean() > winedf['points'].mean()))['country'].unique())



give me list of countries with atleast 100 wine reviews
['Italy' 'Portugal' 'US' 'Spain' 'France' 'Germany' 'Argentina' 'Chile'
 'Australia' 'Austria' 'South Africa' 'New Zealand' 'Israel' 'Hungary'
 'Greece' 'Romania' 'Canada' 'Uruguay' 'Bulgaria']


give me list of countries with above average wine price
['Italy' 'US' 'France' 'Germany' 'Hungary']


give me list of countries with above average wine rating
['Italy' 'US' 'France' 'Germany' 'Australia' 'Austria' 'Israel' 'Hungary'
 'Canada']


In [44]:
#Build Some Top lists for use in further analysis
TopCountriesByPrice = winedf.groupby('country')['price'].agg('mean').sort_values(ascending=False)[:10].index
TopCountriesByCount = winedf.groupby('country')['price'].agg('count').sort_values(ascending=False)[:10].index

TopRegionByPrice = winedf.groupby('country')['price'].agg('mean').sort_values(ascending=False)[:10].index
TopCountriesByCount = winedf.groupby('country')['price'].agg('count').sort_values(ascending=False)[:10].index

TopCountriesByPrice = winedf.groupby('country')['price'].agg('mean').sort_values(ascending=False)[:10].index
TopCountriesByCount = winedf.groupby('country')['price'].agg('count').sort_values(ascending=False)[:10].index

TopCountriesByPrice = winedf.groupby('country')['price'].agg('mean').sort_values(ascending=False)[:10].index
TopCountriesByCount = winedf.groupby('country')['price'].agg('count').sort_values(ascending=False)[:10].index


'''
print(TopCountriesByCount)
print('\n')
print(TopCountriesByPrice)
'''

Index(['US', 'France', 'Italy', 'Spain', 'Portugal', 'Chile', 'Argentina',
       'Austria', 'Australia', 'Germany'],
      dtype='object', name='country')


Index(['Switzerland', 'England', 'France', 'Germany', 'Hungary', 'Italy', 'US',
       'Egypt', 'Canada', 'Australia'],
      dtype='object', name='country')


In [36]:
#wine by winery, country $ top 25 by price
winedf.groupby(['winery','country'])[['price']].mean().price.sort_values(ascending=False)[:25]

winery                    country
Blair                     US         1029.000000
Masseto                   Italy       587.500000
Screaming Eagle           US          500.000000
Krug                      France      397.964628
Vega Sicilia              Spain       367.500000
Contador                  Spain       354.000000
Lokoya                    US          350.000000
Salon                     France      346.428571
Bryant Family             US          335.000000
Yao Ming                  US          327.600000
Dal Forno Romano          Italy       321.277778
Sirena Dunarii            Romania     320.000000
Gaja                      Italy       302.086777
Hundred Acre              US          300.000000
Harlan Estate             US          298.750000
Soldera                   Italy       293.000000
Château Latour            France      283.301053
Château Haut-Brion        France      264.494968
Michel Moritz             France      261.666667
Domaine Henri Rebourseau  France   

In [37]:
# variety by price
winedf.groupby(['variety'])[['price']].mean().price.sort_values(ascending=False)[:20]

variety
Cabernet-Shiraz               150.000000
Muscadel                      141.300000
Mazuelo                        98.500000
Tinto Fino                     83.120000
Mission                        82.571429
Tokay                          82.117647
Champagne Blend                80.512289
Carignan-Syrah                 80.000000
Debit                          72.333333
Picolit                        71.869565
Tokaji                         71.085415
Syrah-Cabernet Franc           69.000000
Bordeaux-style White Blend     68.178423
Cabernet Blend                 66.757377
Nebbiolo                       66.148505
Nasco                          65.000000
Sangiovese Grosso              61.578730
Spätburgunder                  60.092593
Tinta Francisca                60.000000
Bordeaux-style Red Blend       58.384891
Name: price, dtype: float64

In [38]:
winedf.groupby(['region_2', 'country'])[['price']].mean().price.sort_values(ascending=False)[:20]

region_2                      country
Montrachet                    France     601.181818
Romanée-St.-Vivant            France     545.000000
Mazoyeres-Chambertin          France     475.000000
Chevalier-Montrachet          France     438.263158
Grands-Echezeaux              France     391.000000
Musigny                       France     380.000000
Chambertin Clos de Bèze       France     347.250000
Bonnes-Mares                  France     340.333333
Bienvenues Bâtard-Montrachet  France     326.909091
Clos de Tart                  France     319.000000
Bâtard-Montrachet             France     304.888889
Griotte-Chambertin            France     270.000000
Chambertin                    France     269.846154
Criots-Bâtard-Montrachet      France     266.666667
Maury                         France     238.800000
Latricières-Chambertin        France     228.500000
Corton-Rognet                 France     225.000000
Chapelle-Chambertin           France     224.900000
Rivesaltes                

In [39]:
#Top 10 Country By Points
winedf.groupby(['country'])[['points']].mean().points.sort_values(ascending = False)[:10]

country
England     92.888889
Austria     89.276742
France      88.925870
Germany     88.626427
Italy       88.413664
Canada      88.239796
Slovenia    88.234043
Morocco     88.166667
Turkey      88.096154
Portugal    88.057685
Name: points, dtype: float64

In [40]:
#wine by winery, country $ top 25 by points
winedf.groupby(['winery','country'])[['points']].mean().points.sort_values(ascending=False)[:25]

winery                        country  
Sloan                         US           100.000000
Mascarello Giuseppe e Figlio  Italy         99.000000
Domaine des Lambrays          France        98.000000
Château Climens               France        97.111111
Cardinale                     US            97.000000
Gandona                       US            97.000000
Standish                      Australia     97.000000
Au Sommet                     US            97.000000
Ovid                          US            97.000000
Château d'Yquem               France        96.875000
Harlan Estate                 US            96.625000
Château Pétrus                France        96.615385
Screaming Eagle               US            96.000000
Araujo                        US            96.000000
Bryant Family                 US            96.000000
Semper                        US            96.000000
Tenuta dell'Ornellaia         Italy         95.878788
The Maiden                    US          